In [1]:
## Carga de librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
np.random.seed(0)

In [2]:
## Funciones auxiliares
def change_dates(df):
    #changes fecha_inicio_sintomas according to new criteria
    filt_df2 = (df.fecha_inicio_sintomas.isnull()) #filter fecha_inicio_sintomas = inexistant
    n_size = (df[filt_df2]).shape[0]
    df["fecha_inicio_sintomas"] = df["fecha_inicio_sintomas"].fillna(0)
    df["fecha_inicio_sintomas"] = df["fecha_inicio_sintomas"].astype('int')
    df["fecha_apertura"] = df["fecha_apertura"].astype('int')
    b = np.array(df["fecha_apertura"].values)
    a = np.array(df["fecha_inicio_sintomas"].values)
    a[filt_df2.values] = b[filt_df2.values] - np.random.randint(0,9,a[filt_df2.values].shape)
    filt_a = np.where(a<0)
    a[filt_a] = 0
    df = df.drop(["fecha_inicio_sintomas"],axis=1)
    df["fecha_inicio_sintomas"] = a
    return df

In [4]:
tiempo1 = []
tiempo2 = []
tiempo3 = []
N = 1
for cont in range(N):
    print(cont)
    start1 = time.time()
    start2 = time.time()
    ## Carga de datos, sacamos las series temporal
    direccion = '12en20.csv'
    columnas = ['residencia_provincia_nombre','residencia_departamento_nombre','fecha_inicio_sintomas',
                'fecha_apertura','clasificacion_resumen']
    data = pd.read_csv(direccion,sep=",",quotechar='"', usecols=columnas,
                       parse_dates=["fecha_inicio_sintomas","fecha_apertura"],na_values=['']) 
    end2 = time.time()
    data = data[data["clasificacion_resumen"] == "Confirmado"] #filter confirmed cases
    df = pd.DataFrame(data) 
    df = df.drop(['clasificacion_resumen'], axis=1) #drop clasificacion_resumen bc previous filter of confirmed cases
    filt_df1 = (df.fecha_inicio_sintomas.notnull()) #filter fecha_inicio_sintomas = inexistant
    inicio_epidemia = (df.loc[filt_df1,"fecha_inicio_sintomas"]).min() #first symptoms of a person registered
    ultima_actualizacion_sintomas = (df.loc[filt_df1,"fecha_inicio_sintomas"]).max() #last day symptoms of a person registered
    ultima_actualizacion_apertura = (df["fecha_apertura"]).max()
    df["fecha_inicio_sintomas"] -= inicio_epidemia #correcting by inicio_epidemia 
    df["fecha_apertura"] -= inicio_epidemia #correcting by inicio_epidemia
    df.fecha_inicio_sintomas = df.fecha_inicio_sintomas.dt.days #change to int, ditch days 
    df.fecha_apertura = df.fecha_apertura.dt.days #change to int, ditch days
    #print("primer sintoma de persona confirmada: ",inicio_epidemia)
    #print("ultimo sintoma de persona confirmada: ",ultima_actualizacion_sintomas)
    #print("ultima apertura de persona confirmada: ",ultima_actualizacion_apertura)
    df = change_dates(df) # we have replaced all bad or undefined dates
    df = df[["residencia_departamento_nombre","fecha_inicio_sintomas"]]
    df2 = df.groupby(["residencia_departamento_nombre","fecha_inicio_sintomas"]).size().reset_index() 
    df2 = df2.rename(columns={0:'casos'})
    df3 = df2.sort_values(["residencia_departamento_nombre","fecha_inicio_sintomas"],ascending=[True,True])
    #let's list all the provinces that have cases
    #we'll get those times series then, with pd grouping
    provincias = pd.Series(df3["residencia_departamento_nombre"].unique())
    # there are 24 provinces (23 + CABA) + SIN ESPECIFICAR cases
    rows = provincias.shape[0]
    cols = df2["fecha_inicio_sintomas"].max()
    time_series = np.zeros(shape=(rows,cols+1)) #time series of new cases / day
    time_series_ac = np.zeros(shape=(rows,cols+1)) # time series of attack rate / day (attack rate = 14 days cumulate of new cases)
    j = 0
    for i in provincias:
      df_tmp = df3[df3["residencia_departamento_nombre"]==i]
      tmp_dates = np.array(df_tmp["fecha_inicio_sintomas"].values)
      tmp_cases = np.array(df_tmp["casos"].values)
      time_series[j,tmp_dates]= tmp_cases
      j = j + 1
    for i in range(rows):
      for j in range(13,cols):
        time_series_ac[i,j] = np.sum(time_series[i,j-13:j+1])
    # Ahora, lo hacemos en serio, tipo todas las provincias contra todas las provincias, y guardamos los resultados
    start3 = time.time()
    localidades_lagmax = np.zeros(shape=(rows,rows))
    localidades_corrmax = np.zeros(shape=(rows,rows))
    npts = cols
    lags = np.arange(-npts + 1, npts)
    i = 0
    while i < rows:
        j = 0
        prov_tmp_1 = time_series_ac[i,:-10]
        while j < rows:
            prov_tmp_2 = time_series_ac[j,:-10]
            ccov = np.correlate(prov_tmp_1  - prov_tmp_1 .mean(), prov_tmp_2 - prov_tmp_2.mean(), mode='full')
            ccor = ccov / (npts * prov_tmp_1 .std() * prov_tmp_2.std())
            localidades_lagmax[i,j] = lags[np.argmax(ccor)]
            localidades_corrmax[i,j] = np.max(ccor)
            j = j + 1
        i = i +1
    end3 = time.time()
    end1 = time.time()
    tiempo1.append(end1 - start1)
    tiempo2.append(end2 - start2)
    tiempo3.append(end3 - start3)

0


In [5]:
print('Tiempo 1: ',np.array(tiempo1).mean(),np.array(tiempo1).std())
print('Tiempo 2: ',np.array(tiempo2).mean(),np.array(tiempo2).std())
print('Tiempo 3: ',np.array(tiempo3).mean(),np.array(tiempo3).std())

Tiempo 1:  52.541179895401 0.0
Tiempo 2:  18.366300106048584 0.0
Tiempo 3:  30.305973291397095 0.0
